In [1]:
import sqlite3
import pandas as pd

In [3]:
# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

In [5]:
# Tạo bảng student
cursor.execute("""
CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

In [7]:
# Tạo bảng course
cursor.execute("""
CREATE TABLE IF NOT EXISTS course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")


In [9]:
# Xóa dữ liệu cũ nếu có
cursor.execute("DELETE FROM student;")
cursor.execute("DELETE FROM course;")

In [11]:
# Chèn dữ liệu vào bảng course
cursor.executemany("""
INSERT INTO course (id, course_name) VALUES (?, ?);
""", [(12, 'Giai tich'), (34, 'Thong ke'), (26, 'Tin hoc')])

In [13]:
# Chèn dữ liệu vào bảng student
cursor.executemany("""
INSERT INTO student (student_id, name, class, course_id, score) VALUES (?, ?, ?, ?, ?);
""", [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
])
conn.commit()

In [15]:
# 1. Thực hiện JOIN
queries = {
    "Cartesian Product": "SELECT * FROM student, course;",
    "INNER JOIN": "SELECT student.*, course.course_name FROM student INNER JOIN course ON student.course_id = course.id;",
    "LEFT JOIN": "SELECT student.*, course.course_name FROM student LEFT JOIN course ON student.course_id = course.id;",
    "RIGHT JOIN (emulated)": "SELECT student.*, course.course_name FROM course LEFT JOIN student ON student.course_id = course.id;",
    "FULL OUTER JOIN (emulated)": "SELECT student.*, course.course_name FROM student LEFT JOIN course ON student.course_id = course.id UNION SELECT student.*, course.course_name FROM course LEFT JOIN student ON student.course_id = course.id;"
}

for title, query in queries.items():
    print(f"\n{title}:")
    df = pd.read_sql(query, conn)
    print(df)

# Đóng kết nối
conn.close()


Cartesian Product:
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
3            2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
4            2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc
5            2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
6            3       Pham Van Nam  Toan Tin        NaN    7.9  12   Giai tich
7            3       Pham Van Nam  Toan Tin        NaN    7.9  26     Tin hoc
8            3       Pham Van Nam  Toan Tin        NaN    7.9  34    Thong ke
9            4     Le Thanh Huyen  Toan Tin       20.0    7.2  12   Giai tich
10           4     Le Thanh Huyen  Toan Tin       20.0    7.2  26     Tin hoc
11           4     Le Thanh Huyen  Toan Tin 

# 2. Hãy cập nhật những giá trị course_id còn thiếu trong bảng student bằng câu lệnh SQL,
    trong đó các giá trị được điền là những giá trị nằm trong bảng course và loại bỏ những bản ghi tham gia những môn học không tồn tại bảng course. Sau đó hãy cho biết:

- a. Tổng số sinh viên, điểm trung bình của từng lớp.
- b. Tổng số sinh viên, điểm trung bình của từng môn học.
- c. Phân loại thi đua theo số điểm của từng môn học biết:
 
    -i. Điểm TB ≥ 9.0: Xuất sắc.
  
    -ii. 6.0 ≤ Điểm TB ≤ 8.9: Tốt.
  
    -iii. Điểm TB < 6.0: Kém.

In [22]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# 2. Cập nhật course_id còn thiếu và loại bỏ dữ liệu không hợp lệ
cursor.execute("""
UPDATE student
SET course_id = (SELECT id FROM course WHERE course_name = 'Tin hoc')
WHERE course_id IS NULL;
""")

cursor.execute("""
DELETE FROM student
WHERE course_id NOT IN (SELECT id FROM course);
""")
conn.commit()

# 3a. Tính tổng số sinh viên và điểm trung bình theo lớp
query = """
SELECT class, COUNT(*) AS total_students, AVG(score) AS avg_score
FROM student
GROUP BY class;
"""
df = pd.read_sql(query, conn)
print("\nTổng số sinh viên và điểm trung bình theo lớp:")
print(df)

# Đóng kết nối
conn.close()


Tổng số sinh viên và điểm trung bình theo lớp:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000


In [24]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# 3b. Tính tổng số sinh viên và điểm trung bình theo môn học
query = """
SELECT course.course_name, COUNT(student.student_id) AS total_students, AVG(student.score) AS avg_score
FROM student
JOIN course ON student.course_id = course.id
GROUP BY course.course_name;
"""
df = pd.read_sql(query, conn)
print("\nTổng số sinh viên và điểm trung bình theo môn học:")
print(df)

# Đóng kết nối
conn.close()


Tổng số sinh viên và điểm trung bình theo môn học:
  course_name  total_students  avg_score
0   Giai tich               1   6.700000
1    Thong ke               2   9.200000
2     Tin hoc               3   7.366667


In [26]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# 3c. Phân loại thi đua theo số điểm của từng môn học
query = """
SELECT course.course_name, AVG(student.score) AS avg_score,
       CASE 
           WHEN AVG(student.score) >= 9.0 THEN 'Xuất sắc'
           WHEN AVG(student.score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
           ELSE 'Kém'
       END AS ranking
FROM student
JOIN course ON student.course_id = course.id
GROUP BY course.course_name;
"""
df = pd.read_sql(query, conn)
print("\nPhân loại thi đua theo số điểm của từng môn học:")
print(df)

# Đóng kết nối
conn.close()


Phân loại thi đua theo số điểm của từng môn học:
  course_name  avg_score   ranking
0   Giai tich   6.700000       Tốt
1    Thong ke   9.200000  Xuất sắc
2     Tin hoc   7.366667       Tốt


# 3. Hãy xếp hạng sinh viên thông qua:
a. Điểm số.

b. Điểm số theo lớp học.

c. Điểm số theo mã môn học.
và cho biết top 3 sinh viện đạt thứ hạng cao nhất, top 3 sinh viên đạt thứ hạng thấp nhất theo từng trường hợp trên.

In [46]:
# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# a. Xếp hạng theo điểm số
query = """
SELECT student_id, name, score,
       RANK() OVER (ORDER BY score DESC) AS rank
FROM student;
"""
df = pd.read_sql(query, conn)
print("\nXếp hạng sinh viên theo điểm số:")
print(df)

# Đóng kết nối
conn.close()


Xếp hạng sinh viên theo điểm số:
   student_id               name  score  rank
0           2       Tran Thi Lan    9.2     1
1           7      Bui Tien Dung    9.2     1
2           3       Pham Van Nam    7.9     3
3           9     Duong Huu Phuc    7.2     4
4          10       Cao Thi Hanh    7.0     5
5           1  Nguyen Minh Hoang    6.7     6


In [42]:
# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# b. Xếp hạng theo điểm số trong từng lớp
query = """
SELECT student_id, name, class, score,
       RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank
FROM student;
"""
df = pd.read_sql(query, conn)
print("\nXếp hạng sinh viên theo điểm số trong từng lớp:")
print(df)

# Đóng kết nối
conn.close()


Xếp hạng sinh viên theo điểm số trong từng lớp:
   student_id               name     class  score  rank
0           2       Tran Thi Lan   Kinh Te    9.2     1
1           7      Bui Tien Dung   Kinh Te    9.2     1
2           9     Duong Huu Phuc   Kinh Te    7.2     3
3          10       Cao Thi Hanh  May Tinh    7.0     1
4           1  Nguyen Minh Hoang  May Tinh    6.7     2
5           3       Pham Van Nam  Toan Tin    7.9     1


In [44]:
# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# c. Xếp hạng theo điểm số trong từng môn học
query = """
SELECT student_id, name, course_id, score,
       RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank
FROM student;
"""
df = pd.read_sql(query, conn)
print("\nXếp hạng sinh viên theo điểm số trong từng môn học:")
print(df)

# Đóng kết nối
conn.close()


Xếp hạng sinh viên theo điểm số trong từng môn học:
   student_id               name  course_id  score  rank
0           1  Nguyen Minh Hoang         12    6.7     1
1           3       Pham Van Nam         26    7.9     1
2           9     Duong Huu Phuc         26    7.2     2
3          10       Cao Thi Hanh         26    7.0     3
4           2       Tran Thi Lan         34    9.2     1
5           7      Bui Tien Dung         34    9.2     1


In [48]:
# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# d. Top 3 sinh viên đạt thứ hạng cao nhất và thấp nhất theo từng tiêu chí
for criteria in ["score", "class", "course_id"]:
    query = f"""
    SELECT student_id, name, {criteria}, score,
           RANK() OVER (PARTITION BY {criteria} ORDER BY score DESC) AS rank
    FROM student;
    """
    df = pd.read_sql(query, conn)
    print(f"\nTop 3 sinh viên đạt điểm cao nhất theo {criteria}:")
    print(df[df['rank'] <= 3])
    
    query = f"""
    SELECT student_id, name, {criteria}, score,
           RANK() OVER (PARTITION BY {criteria} ORDER BY score ASC) AS rank
    FROM student;
    """
    df = pd.read_sql(query, conn)
    print(f"\nTop 3 sinh viên đạt điểm thấp nhất theo {criteria}:")
    print(df[df['rank'] <= 3])

# Đóng kết nối
conn.close()


Top 3 sinh viên đạt điểm cao nhất theo score:
   student_id               name  score  score  rank
0           1  Nguyen Minh Hoang    6.7    6.7     1
1          10       Cao Thi Hanh    7.0    7.0     1
2           9     Duong Huu Phuc    7.2    7.2     1
3           3       Pham Van Nam    7.9    7.9     1
4           2       Tran Thi Lan    9.2    9.2     1
5           7      Bui Tien Dung    9.2    9.2     1

Top 3 sinh viên đạt điểm thấp nhất theo score:
   student_id               name  score  score  rank
0           1  Nguyen Minh Hoang    6.7    6.7     1
1          10       Cao Thi Hanh    7.0    7.0     1
2           9     Duong Huu Phuc    7.2    7.2     1
3           3       Pham Van Nam    7.9    7.9     1
4           2       Tran Thi Lan    9.2    9.2     1
5           7      Bui Tien Dung    9.2    9.2     1

Top 3 sinh viên đạt điểm cao nhất theo class:
   student_id               name     class  score  rank
0           2       Tran Thi Lan   Kinh Te    9.2     1
1   

# 4. Hãy bổ sung thêm một trường graduation_date có kiểu dữ liệu là DATETIME vào bảng student để xác định thời gian tốt nghiệp của từng bạn, trong đó thời gian tốt nghiệp được xác định bởi thời gian hiện tại cộng với số hạng tương ứng của bạn đó tính theo điểm số.

In [52]:
# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# 4. Thêm cột graduation_date vào bảng student
cursor.execute("""
ALTER TABLE student ADD COLUMN graduation_date DATETIME;
""")

# Cập nhật ngày tốt nghiệp dựa trên thứ hạng theo điểm số
query = """
UPDATE student
SET graduation_date = DATETIME('now', '+' || (
    SELECT RANK() OVER (ORDER BY score DESC) FROM student s WHERE s.student_id = student.student_id
) || ' days');
"""
cursor.execute(query)
conn.commit()

# Kiểm tra kết quả
query = "SELECT student_id, name, score, graduation_date FROM student;"
df = pd.read_sql(query, conn)
print("\nDanh sách sinh viên với ngày tốt nghiệp:")
print(df)

# Đóng kết nối
conn.close()


Danh sách sinh viên với ngày tốt nghiệp:
   student_id               name  score      graduation_date
0           1  Nguyen Minh Hoang    6.7  2025-03-12 09:18:23
1           2       Tran Thi Lan    9.2  2025-03-12 09:18:23
2           3       Pham Van Nam    7.9  2025-03-12 09:18:23
3           7      Bui Tien Dung    9.2  2025-03-12 09:18:23
4           9     Duong Huu Phuc    7.2  2025-03-12 09:18:23
5          10       Cao Thi Hanh    7.0  2025-03-12 09:18:23
